# Decorators

## Simple example

A **decorator** is a function that takes in a function (and usually runs it). You can think of them like wrappers that add functionality to a function that already exists.

In [1]:
user = {'username': 'jose123', 'access_level': 'admin'}

def user_has_permission(func):
    def secure_func():
        if user.get('access_level') == 'admin':
            return func()
    return secure_func
    
def my_function():
    return 'Password for admin panel is 1234.'

my_secure_function = user_has_permission(my_function)

print(my_secure_function())

Password for admin panel is 1234.


## @syntax

By using `@user_has_permission` syntax, we get the same results as explicitly passing a function to another function. In essence, we are setting up the association that `user_has_permission()` will accept `my_function()` as an argument:

In [4]:
user = {'username': 'jose123', 'access_level': 'admin'}


def user_has_permission(func):
    def secure_func():
        if user.get('access_level') == 'admin':
            return func()
    return secure_func

@user_has_permission
def my_function():
    """I am a docstring."""
    return 'Password for admin panel is 1234.'


print(my_function())

Password for admin panel is 1234.


In [7]:
print(my_function.__name__)
print(my_function.__doc__)

secure_func
None


Note though that `my_function()` is returning information about `secure_func()` because it has been replaced. The next section covers how to fix this.

In [8]:
@user_has_permission
def another():
    pass

print(my_function.__name__)
print(another.__name__)

secure_func
secure_func


## `functools` wraps

A decorator called `functools` wraps around the function to let python know what the original function is wrapping around another:

In [9]:
import functools

user = {'username': 'jose123', 'access_level': 'guest'}


def user_has_permission(func):
    @functools.wraps(func)
    def secure_func():
        if user.get('access_level') == 'admin':
            return func()
    return secure_func

@user_has_permission
def my_function():
    """
    Allows us to retrieve the password for the admin panel.
    """
    return 'Password for admin panel is 1234.'


@user_has_permission
def another():
    pass

print(my_function.__name__)
print(another.__name__)

my_function
another


## Decorating with parameters

Passing parameters is like most other functions, but note that `user_has_permission()` doesn't need to include the parameter in its definition:

In [10]:
import functools

user = {'username': 'jose123', 'access_level': 'admin'}


def user_has_permission(func):
    @functools.wraps(func)
    def secure_func(panel):
        if user.get('access_level') == 'admin':
            return func(panel)
    return secure_func

@user_has_permission
def my_function(panel):
    """
    Allows us to retrieve the password for the admin panel.
    """
    return f'Password for {panel} panel is 1234.'


print(my_function.__name__)
print(my_function('movies'))

my_function
Password for movies panel is 1234.


However, it is recognizing and passing the parameter, because trying to use it on another function produces and error:

In [11]:
@user_has_permission
def another():
    return 'Hello'

print(another())

TypeError: secure_func() missing 1 required positional argument: 'panel'

In [14]:
def third_level(access_level):
    def user_has_permission(func):
        @functools.wraps(func)
        def secure_func(panel):
            if user.get('access_level') == 'admin':
                return func(panel)
        return secure_func
    return user_has_permission

@third_level('user')
def my_function(panel):
    """
    Allows us to retrieve the password for the admin panel.
    """
    return f'Password for {panel} panel is 1234.'


print(my_function.__name__)
print(my_function('movies'))

my_function
Password for movies panel is 1234.


## Functions that accept multiple arguments

`*args` is the convention to allow passing a flexible number of positional arguments, and `**kwargs` allows passing a flexible number of named arguments (key-value pairs).

Each of the below is equivalent, but increasingly more pythonic:

In [16]:
def add_all(a1, a2, a3, a4):
    return a1+a2+a3+a4

print(add_all(1, 3, 5, 7))

16


In [17]:
def add_all(a1, a2, a3, a4):
    return a1+a2+a3+a4

vals = (1, 3, 5, 7)

print(add_all(*vals))

16


In [18]:
def add_all(*args):
    return sum(args)

print(add_all(*vals))

16


In [20]:
def pretty_print(**kwargs):
    for k, v in kwargs.items():
        print(f'For {k} we have {v}.')

In [21]:
pretty_print(username='jose123', access_level='admin')

For username we have jose123.
For access_level we have admin.


In [22]:
pretty_print(**{'username': 'jose123', 'access_level': 'admin'})

For username we have jose123.
For access_level we have admin.


## Generic decorators

In the below example, we are ensuring we can pass all arguments through the rest of the decorator, making it generic for any function that it affects.

In [23]:
import functools

user = {'username': 'jose123', 'access_level': 'admin'}


def user_has_permission(func):
    @functools.wraps(func)
    def secure_func(*args, **kwargs):
        if user.get('access_level') == 'admin':
            return func(*args, **kwargs)
    return secure_func

@user_has_permission
def my_function(panel):
    """
    Allows us to retrieve the password for the admin panel.
    """
    return f'Password for {panel} panel is 1234.'


@user_has_permission
def another():
    pass


print(my_function.__name__)

print(my_function('movies'))
print(another())

my_function
Password for movies panel is 1234.
None


A final note, you can apply multiple decorators on one function. They evaluate top-down.